In [1]:
## Generate chatbot with message history using prompt templates
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)

prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant that helps people find information."),
    MessagesPlaceholder(variable_name="messages")])

chain=prompt|model

In [2]:
chain.invoke({"messages":[HumanMessage(content="Hi I am AI Engineer")]})

AIMessage(content='Hello AI Engineer. How can I assist you today? Are you looking for information on a specific AI-related topic, such as machine learning, deep learning, or natural language processing? Or perhaps you need help with a particular AI project or technology?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 51, 'total_tokens': 101, 'completion_time': 0.058842952, 'prompt_time': 0.002464303, 'queue_time': 0.049253107, 'total_time': 0.061307255}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--05f53e4b-32f5-4c1b-8be7-14c002a0e0ee-0', usage_metadata={'input_tokens': 51, 'output_tokens': 50, 'total_tokens': 101})

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory ## ChatMessageHistory will be used to store the chat history
from langchain_core.chat_history import BaseChatMessageHistory ## BaseChatMessageHistory is the base class for chat history.It is used to define the type of chat history.For example, we can use it to define the type of chat history as ChatMessageHistory.
from langchain_core.runnables import RunnableWithMessageHistory ## RunnableWithMessageHistory is used to create a runnable with message history.It is used to create a runnable that can store the chat history.

store={} ## store is a dictionary that will be used to store the chat history for different sessions.Dictionary here in simple words is a key value pair where key is session id and value is chat history.
def get_session_history(session_id: str) -> BaseChatMessageHistory: ## get_session_history is a function that takes session_id as input and returns the chat history for that session_id.
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(chain,get_session_history) 

config={"configurable":{"session_id":"user_1234"}}

response=with_message_history.invoke([HumanMessage(content="Hi I am AI Engineer")],config=config)

response

AIMessage(content="As an AI Engineer, you likely work on the development and implementation of artificial intelligence (AI) systems, including machine learning algorithms, natural language processing, and computer vision. \n\nWhat can I help you with today? Are you working on a specific project or do you have a question about AI engineering? Perhaps you need help with:\n\n- Choosing a suitable machine learning algorithm\n- Implementing a deep learning model\n- Designing a natural language processing system\n- Optimizing AI model performance\n- Something else?\n\nPlease let me know, and I'll do my best to provide you with helpful information and insights.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 51, 'total_tokens': 174, 'completion_time': 0.184206559, 'prompt_time': 0.002477342, 'queue_time': 0.050783208, 'total_time': 0.186683901}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'o

In [4]:
## Add more complexity to the chat.It will respond as per the language provided by the user.

prompt=ChatPromptTemplate.from_messages([ ("system","You are a helpful assistant that helps people find information."
"Answer all the question to the best of your ability in {language}"),
MessagesPlaceholder(variable_name="messages")])

chain=prompt|model

In [5]:
response=chain.invoke({"messages":[HumanMessage(content="Hi I am AI Engineer")],"language":"Hindi"})
response.content

'नमस्ते! एक AI इंजीनियर के रूप में, आप हमेशा नवीनतम तकनीकों और तकनीकी जानकारी की तलाश में रहते हैंग। मैं आपकी सहायता के लिए यहाँ हूँ। क्या आपको कोई विशिष्ट जानकारी या समस्या का समाधान चाहिए?'

In [12]:
## Lets now wrap this more complicated chain in a Message History Class.Since there are multiple keys in the input we need to specify the correct key to use for save the chat history.

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages")

In [13]:
config2={"configurable":{"session_id":"user_12345"}}

response=with_message_history.invoke({'messages':[HumanMessage(content="Hi I am AI Engineer")],"language":"Hindi"},config=config2)

response.content

'safe'

In [14]:
response=with_message_history.invoke({'messages':[HumanMessage(content="What is my job")],"language":"Hindi"},config=config2)

response.content

'safe'